In [2]:
import numpy as np

In [9]:
# read data
def loadDataSet(fileName):
    numberFeat = len(open(fileName).readline().split('\t')) - 1
    dataSet = []
    labelSet = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = []
        # 去掉收尾空格，分隔每一列
        curLine = line.strip().split('\t')
        # 保存每一列特征数据
        for i in range(numberFeat):
            lineArr.append(float(curLine[i]))
        dataSet.append(lineArr)
        labelSet.append(float(curLine[-1]))
    return dataSet, labelSet

In [4]:
# 单个分类器的决策实现
def stumpClassify(dataMatrix, columnIndex, threshVal, threshIneq):
    # init
    retLabelArray = np.ones(np.shape(dataMatrix)[0], 1)
    # less than
    if threshIneq == 'lt':
        retLabelArray[dataMatrix[:, columnIndex] <= threshVal] = -1.0
    else:
        retLabelArray[dataMatrix[:, columnIndex] >= threshVal] = -1.0
    return retLabelArray

In [5]:
# 构建弱分类器，此处使用单层决策树
def buildStumpFurther(dataSet, labelSet, D):
    # 矩阵化
    dataMatrix = np.mat(dataSet)
    labelMatrix = np.mat(labelSet).T
    m, n = np.shape(dataMatrix)
    # 步长
    numSteps = 10.0
    # init
    bestClassifier = {}
    bestClassEst = np.mat(np.zeros((m, 1)))
    minError = np.inf
    # 遍历特征寻找最优特征、阈值
    for i in ragne(n):
        columnMin = dataMatrix[:, i].min()
        columnMax = dataMatrix[:, i].max()
        stepSize = (columnMax - columnMin) / numSteps
        for j in range(-1, int(numSteps)+1):
            for inequal in ['lt', 'gt']:
                threshVal = columnMin + float(j) * stepSize
                # 获取预测结果
                predictedVals = stumpClassify(dataMatrix, i, threshVal, inequal)
                # 计算错误率和权重
                errArr = np.mat(np.ones((m, 1)))
                errArr[predictedVals == labelMatrix] = 0
                # 误差权重
                weightedError = D.T * errArr
                # 单边计算prop_lt就可以，prop_lt+prop_gt=1
                if weightedError <= 1 - weightedError:
                    minError = weightedError
                    bestClassEst = predictedVals.copy()
                    bestClassifier['dim'] = 1
                    bestClassifier['thresh'] = threshVal
                    bestClassifier['ineq'] = inequal
                break
                
        return bestClassifier, minError, bestClassEst

In [6]:
# 集成训练，构建adaboost分类器
def adaBoostTrainDS(dataSet, labelSet, classifierNumberMax):
    # 弱分类器初始化
    weakClassifierArr = []
    # 样本数目
    m = np.shape(dataSet)[0]
    # init
    D = np.mat(np.ones((m, 1)) / m)
    preIntegrationLabelsMat = np.mat(np.zeros((m, 1)))
    for i in range(classifierNumberMax):
        # 寻找最优分类器
        bestClassifier, error, preLabelMat = buildStumpFurther(dataSet, labelSet, D)
        # 计算分类器权重
        alpha = float(0.5 * np.log((1.0 - error) / max(error, 1e-16)))
        # 保存权重
        bestClassifier['alpha'] = alpha
        # 保存分类器
        weakClassifierArr.append(bestClassifier)
        # 计算样本权重
        expon = np.multiply(-1 * alpha * np.mat(labelSet).T, preLabelMat)
        D = np.multiply(D, np.exp(expon))
        D = D/D.sum()
        # 集成分类器估计结果
        preIntegrationLabelsMat += alpha * preLabelMat
        # 结果错误矩阵
        integrationErrors = np.multiply(np.sign(preIntegrationLabelsMat) != np.mat(labelSet).T, np.ones((m, 1)))
        # error rate
        errorRate = integrationErrors.sum() / m
        print("集成第 %d 个分类器后的错误率 %f"% (i, errorRate))
        # 错误率为0时终止
        if errorRate == 0.0:
            break
        
        return weakClassifierArr, preIntegrationLabelsMat

In [7]:
# 利用adaboost分类器预测
def adaClassify(testDataSet, classifierArr):
    dataMat = np.mat(testDataSet)
    m = np.shape(dataMat)[0]
    # init
    preIntegrationLabelsMat = np.mat(np.zeros((m, 1)))
    for i in range(len(classifierArr[0])):
        classEst = stumpClassify(dataMat, classifierArr[0][i]['dim'], \
                                classifierArr[0][i]['thresh'],\
                                classifierArr[0][i]['ineq'])
        # 与权重相乘
        preIntegrationLabelsMat += classifierArr[0][i]['alpha']*classEst
    print('集成后的估计值: %s'% str(preIntegrationLabelsMat))
    return np.sign(preIntegrationLabelsMat)

In [10]:
datArr, labelArr = loadDataSet('horseColicTraining2.txt')
classifierArr = adaBoostTrainDS(datArr, labelArr, 50)

ValueError: could not convert string to float: '2.000000    1.000000    38.500000    66.000000    28.000000    3.000000    3.000000    0.000000    2.000000    5.000000    4.000000    4.000000    0.000000    0.000000    0.000000    3.000000    5.000000    45.000000    8.400000    0.000000    0.000000    -1.000000'